<a href="https://www.kaggle.com/code/samithsachidanandan/image-classification-cnn-in-pytorch?scriptVersionId=283713896" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

### Import Libraries 

In [1]:
import numpy as np 
from PIL import Image

import torch 
import torch.nn as nn 
import torch.nn.functional as F
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms



### Converting an image to a PyTorch tensor

In [2]:
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616))
])

test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616))
])

### Loading the Dataset

In [3]:
   import os
   data_dir = '/kaggle/working/data'
   os.makedirs(data_dir, exist_ok=True)

In [5]:
train_data = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=train_transform)
test_data = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=test_transform)

train_loader = torch.utils.data.DataLoader(train_data, batch_size=32, shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=32, shuffle=False, num_workers=2)

100%|██████████| 170M/170M [00:17<00:00, 9.59MB/s] 


In [6]:
image, label = train_data[0]

In [7]:
image.size()

torch.Size([3, 32, 32])

In [8]:
class_names = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']


### Neural Network Architecture

In [9]:
class NeuralNet(nn.Module):
    def __init__(self):
        super().__init__()
        
    
        self.conv1 = nn.Conv2d(3, 64, 3, padding=1)
        self.bn1 = nn.BatchNorm2d(64)
        self.conv2 = nn.Conv2d(64, 64, 3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.dropout1 = nn.Dropout2d(0.2)
        
   
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.conv4 = nn.Conv2d(128, 128, 3, padding=1)
        self.bn4 = nn.BatchNorm2d(128)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.dropout2 = nn.Dropout2d(0.3)
        

        self.conv5 = nn.Conv2d(128, 256, 3, padding=1)
        self.bn5 = nn.BatchNorm2d(256)
        self.conv6 = nn.Conv2d(256, 256, 3, padding=1)
        self.bn6 = nn.BatchNorm2d(256)
        self.pool3 = nn.MaxPool2d(2, 2)
        self.dropout3 = nn.Dropout2d(0.4)
        
     
        self.fc1 = nn.Linear(256 * 4 * 4, 512)
        self.bn7 = nn.BatchNorm1d(512)
        self.dropout4 = nn.Dropout(0.5)
        self.fc2 = nn.Linear(512, 256)
        self.bn8 = nn.BatchNorm1d(256)
        self.dropout5 = nn.Dropout(0.5)
        self.fc3 = nn.Linear(256, 10)

    def forward(self, x):
      
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = self.pool1(x)
        x = self.dropout1(x)
        
     
        x = F.relu(self.bn3(self.conv3(x)))
        x = F.relu(self.bn4(self.conv4(x)))
        x = self.pool2(x)
        x = self.dropout2(x)
        
 
        x = F.relu(self.bn5(self.conv5(x)))
        x = F.relu(self.bn6(self.conv6(x)))
        x = self.pool3(x)
        x = self.dropout3(x)
        
 
        x = torch.flatten(x, 1)
        x = F.relu(self.bn7(self.fc1(x)))
        x = self.dropout4(x)
        x = F.relu(self.bn8(self.fc2(x)))
        x = self.dropout5(x)
        x = self.fc3(x)
        
        return x
 

In [10]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Using device: cuda


In [11]:
net = NeuralNet().to(device)
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001, weight_decay=1e-4)

In [12]:
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5, verbose=True)

/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


### Training 

In [13]:
best_accuracy = 0.0

for epoch in range(100):
    print(f'Epoch {epoch+1}/100')
    
  
    net.train()
    running_loss = 0.0
    
    for i, data in enumerate(train_loader):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    avg_train_loss = running_loss / len(train_loader)
    
   
    net.eval()
    correct = 0
    total = 0
    val_loss = 0.0
    
    with torch.no_grad():
        for data in test_loader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = net(images)
            loss = loss_function(outputs, labels)
            val_loss += loss.item()
            
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    avg_val_loss = val_loss / len(test_loader)
    accuracy = 100 * correct / total
    
    print(f'Train Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f} | Accuracy: {accuracy:.2f}%')
    
 
    scheduler.step(avg_val_loss)

Epoch 1/100
Train Loss: 1.6949 | Val Loss: 1.1868 | Accuracy: 56.52%
Epoch 2/100
Train Loss: 1.3280 | Val Loss: 0.9780 | Accuracy: 65.35%
Epoch 3/100
Train Loss: 1.1634 | Val Loss: 0.8573 | Accuracy: 69.81%
Epoch 4/100
Train Loss: 1.0529 | Val Loss: 0.7242 | Accuracy: 75.09%
Epoch 5/100
Train Loss: 0.9739 | Val Loss: 0.6709 | Accuracy: 76.97%
Epoch 6/100
Train Loss: 0.9177 | Val Loss: 0.6370 | Accuracy: 78.06%
Epoch 7/100
Train Loss: 0.8730 | Val Loss: 0.5946 | Accuracy: 79.59%
Epoch 8/100
Train Loss: 0.8376 | Val Loss: 0.5659 | Accuracy: 80.62%
Epoch 9/100
Train Loss: 0.8019 | Val Loss: 0.5413 | Accuracy: 81.67%
Epoch 10/100
Train Loss: 0.7718 | Val Loss: 0.5483 | Accuracy: 81.15%
Epoch 11/100
Train Loss: 0.7508 | Val Loss: 0.5042 | Accuracy: 83.20%
Epoch 12/100
Train Loss: 0.7316 | Val Loss: 0.4967 | Accuracy: 83.18%
Epoch 13/100
Train Loss: 0.7117 | Val Loss: 0.4726 | Accuracy: 83.94%
Epoch 14/100
Train Loss: 0.7034 | Val Loss: 0.4722 | Accuracy: 84.11%
Epoch 15/100
Train Loss: 0.68

In [14]:
torch.save(net.state_dict(), 'trained_net.pth')

In [15]:
net = NeuralNet()
net.load_state_dict(torch.load('trained_net.pth'))

<All keys matched successfully>